# QBUS3850: Lab 2 Tasks

The data set CBA_1991-2017.csv (dataset could found on Canvas) contains the monthly closing stock prices stock prices of Commonwealth Bank of Australia (CBA) from August 1991 to December 2017.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

## 1. Load the data and plot the original data curve.

In [ ]:
data = pd.read_csv('CBA_1991-2017.csv', parse_dates=['Date'], index_col='Date')

# TODO: visualise the data

## 2. Write Python script to do a $2 \times 12$-MA (Moving average of moving average) method and plot them. You may call pandas package’s rolling_mean or rolling function to do the $2 \times 12$ smoothing. Then plot the smoothing results onto the original time-series data curve.

In [ ]:
from pandas.api.indexers import FixedForwardWindowIndexer

indexer = FixedForwardWindowIndexer(window_size=2)
# TODO: Implements rolling average

## 3. Do a $5$-MA and compare the results with $2 \times 12$-MA. You may plot these 2 MA results together and examine what is the difference.

In [ ]:
Trend2 = data.rolling(5,center = True).mean()

# TODO: Implements 5-MA rolling average

### 4. Carry out an additive classical decomposition on the log series

In [ ]:
# TODO: find the log of the series

In [ ]:
## Construct the trend and residual.
trend = ts_log.rolling( 12, center = True).mean().rolling( 2, center = True ).mean().shift(-1)
# TODO: find residual

## TODO: Visual check for stationarity

## Dickey-Fuller test for stationarity
from statsmodels.tsa.stattools import adfuller
# TODO

In [ ]:
## Detrend data
ts_res = ts_log - trend
#print( ts_res[:12] )
#print( ts_res[-12:] )

## Remove incomplete years and reshape array. There are 24 years of complete yearly data.
ts_res_clean = np.nan_to_num( ts_res[17:-12] )
monthly_S = np.reshape( ts_res_clean, (24,12) )

## Each of the 12 columns of monthly_S contains data for a given month.
print( monthly_S[0,:] )
print( monthly_S[23,:] )


In [ ]:
## Compute seasonal indices
monthly_avg = np.mean(monthly_S, axis=0)

## De-mean seasonal indices
mean_allmonth = monthly_avg.mean()
monthly_avg_normalized = monthly_avg - mean_allmonth
plt.plot( monthly_avg_normalized )
assert( np.isclose( monthly_avg_normalized.mean(), 0.0 ) )

In [ ]:
## Repeat the seasonal indices over 24 years
# TODO: use np tile function

## Seasonally adjusted = original - seasonal indices
seasonally_adjusted = ts_log[17:-12] - tiled_avg

## Update the trend-cycle
trend_final = seasonally_adjusted.rolling(12, center =True).mean().rolling(2, center = True).mean().shift(-1)
ts_res = ts_log[17:-12] - trend_final 

plt.figure()
# TODO: plot subpolts of the seasonality trend


### 5. Compare results with X13 decomposition.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

from statsmodels.tsa import x13

In [ ]:
data = pd.read_csv('CBA_1991-2017.csv')
print( data.head() )
print( data.dtypes )

data['Date']=pd.to_datetime(data['Date'], dayfirst=True)
data.set_index('Date', inplace=True)
print( data.head() )
print( data.dtypes )


In [ ]:
model = x13.x13_arima_analysis( data['Close'], log=True, forecast_periods=12 )
print( model.results )

In [ ]:
plt.figure()
fig, ax = plt.subplots(5, 1, figsize = (15,12))
ax[0].set_title( 'CBA X13')
ax[0].plot( model.observed )
ax[1].plot( model.observed / model.seasadj )
ax[2].plot( model.seasadj )
ax[3].plot( model.trend )
ax[4].plot( model.irregular )
ax[0].legend(['observed'], loc=2)
ax[1].legend(['seasonality'], loc=2)
ax[2].legend(['seas_adj'], loc=2)
ax[3].legend(['trend'], loc=2)
ax[4].legend(['irregular'], loc=2)
fig.subplots_adjust(hspace=0.5)

#plt.savefig('CBA-X13as.png')
